In [1]:
!git clone https://github.com/JustAnotherArchivist/snscrape

Cloning into 'snscrape'...


In [2]:
!pip install snscrape

In [3]:
!snscrape 

usage: snscrape [-h] [--version] [--citation] [-v] [--dump-locals] [--retry N]
                [-n N] [-f FORMAT | --jsonl] [--with-entity]
                [--since DATETIME] [--progress]
                SCRAPER ...
snscrape: error: the following arguments are required: SCRAPER
